In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import MinMaxScaler, StandardScaler # Normalization - Standardization
from sklearn import preprocessing
from sklearn.utils import resample
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPClassifier
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTEENN
import matplotlib.pyplot as plt
from sklearn.feature_selection import VarianceThreshold
from sklearn.ensemble import RandomForestClassifier
from imblearn.pipeline import Pipeline

In [2]:
us = pd.read_csv('../data/users.csv')
dev = pd.read_csv('../data/devices.csv')

tr1 = pd.read_csv('../data/transactions_1.csv')
tr2 = pd.read_csv('../data/transactions_2.csv')
tr3 = pd.read_csv('../data/transactions_3.csv')

trans = pd.concat([tr1, tr2, tr3])
notif = pd.read_csv('../data/notifications.csv')

In [3]:
us_dev = pd.merge(us, dev, on='user_id', how='inner')
us_dev['age'] = 2020 - us_dev['birth_year']
us_dev['brand'] = us_dev['brand'].str.replace('Unknown', 'Android')

brand_dict = {'Android': 0, 'Apple': 1}
us_dev['brand'] = us_dev['brand'].map(brand_dict)
# us_dev['brand'].value_counts()

brand_dict1 = {'STANDARD': 0, 'SILVER': 1, 'GOLD': 1}
us_dev['plan'] = us_dev['plan'].map(brand_dict1)
# us_dev['brand'].value_counts()
us_dev.head(5)
us_dev.shape
# userdf['brand'] = userdf['brand'].str.replace('Unknown', 'Android')

(15544, 14)

In [4]:
def change_country(x):     
  if x=='AT' or x=='BE' or x=='DE' or x=='LU' or x=='NL' or x=='LI' or x=='CH':
    return 'Central_Rich_Europe'
  elif x=='IE' or x=='GB' or x=='GG' or x=='IM' or x=='JE' or x=='GI' or x=='DK' or x=='NO' or x=='SE' or x=='FI' or x=='IS':
    return 'North_Europe_Scand'
  elif x=='FR' or x=='RE' or x=='GP' or x=='MQ' or x=='MT' or x=='IT' or x=='CY' or x=='ES' or x=='PT':
    return 'Mediteranian_Europe'
  elif x=='EE' or x=='SK' or x=='HU' or x=='LV' or x=='SI' or x=='BG' or x=='GR' or x=='HR' or x=='LT' or x=='RO' or x=='PL' or x=='CZ':
    return 'East_Europe_Balkans'
  elif x=='AU':
    return 'Australia'

In [5]:
us_dev['country'] = us_dev['country'].apply(change_country)
us_dev = pd.concat([pd.get_dummies(us_dev['country']), us_dev], axis=1)
us_dev.head(5)

,Australia,Central_Rich_Europe,East_Europe_Balkans,Mediteranian_Europe,North_Europe_Scand,user_id,birth_year,country,city,created_date,user_settings_crypto_unlocked,plan,attributes_notifications_marketing_push,attributes_notifications_marketing_email,num_contacts,num_referrals,num_successful_referrals,brand,age
0,0,0,0,1,0,user_0,1992,Mediteranian_Europe,Madrid,2018-09-10 18:46:42.843855,0,0,1.0,1.0,3,0,0,1,28
1,0,0,1,0,0,user_1,1985,East_Europe_Balkans,Klaipeda,2018-01-11 18:53:10.531146,1,0,NaN,NaN,75,0,0,1,35
2,0,0,0,0,1,user_2,1996,North_Europe_Scand,Carrickmacross,2018-11-29 17:29:16.879324,0,0,NaN,NaN,71,0,0,1,24
3,0,0,0,0,1,user_3,1981,North_Europe_Scand,Dublin,2018-02-17 11:58:08.319664,0,0,NaN,NaN,219,0,0,1,39
4,0,0,0,0,1,user_4,1992,North_Europe_Scand,London,2018-01-08 18:40:21.097856,0,0,NaN,NaN,0,0,0,1,28


In [6]:
notif.reason.value_counts()
print(dev.user_id.shape)
print(us.user_id.shape)
print(us_dev.user_id.shape)
us_dev.isna().sum()

(15544,)
(15544,)
(15544,)


Australia                                      0
Central_Rich_Europe                            0
East_Europe_Balkans                            0
Mediteranian_Europe                            0
North_Europe_Scand                             0
user_id                                        0
birth_year                                     0
country                                        0
city                                           0
created_date                                   0
user_settings_crypto_unlocked                  0
plan                                           0
attributes_notifications_marketing_push     5260
attributes_notifications_marketing_email    5260
num_contacts                                   0
num_referrals                                  0
num_successful_referrals                       0
brand                                          0
age                                            0
dtype: int64

In [7]:
trans = trans[(trans.transactions_state == 'COMPLETED') & (trans.amount_usd>0)]
trans['created_date'] = pd.to_datetime(trans['created_date'])
trans.transactions_type.value_counts()

CARD_PAYMENT    999411
TRANSFER        395673
TOPUP           242620
EXCHANGE        124764
ATM              61518
CASHBACK         54499
FEE              18757
CARD_REFUND       9207
TAX               2242
REFUND            1193
Name: transactions_type, dtype: int64

In [8]:
trans_grouped = trans.groupby(['user_id'], as_index=True)['amount_usd'].agg(['sum','count','mean'])
trans_grouped.rename(columns={'sum':'total_amount', 'count':'number_transactions','mean':'avg_amount'}, inplace=True)
# trans_grouped.info()


trans_grouped2 = trans.groupby(['user_id'], as_index=True)['created_date'].agg(['min','max'])
trans_grouped2.rename(columns={'min':'first_transaction', 'max':'last_transaction'}, inplace=True)
# trans_grouped2.info()


trans_grouped2['transaction_period']=( (trans_grouped2['last_transaction']-trans_grouped2['first_transaction']) ).dt.days
# print(trans_grouped2.head(4))
# trans_grouped2['transaction_period']=trans_grouped2['transaction_period'].dt.days
#['transaction_period'].days
# print(trans_grouped2.head(4))

trans_grouped_all = pd.merge(trans_grouped, trans_grouped2, on='user_id', how='inner')
trans_grouped_all.head(5)

,total_amount,number_transactions,avg_amount,first_transaction,last_transaction,transaction_period
user_id,,,,,,
user_0,6925.08,520,13.317462,2018-09-10 18:54:08.965855,2019-05-10 05:09:13.868855,241
user_1,12955.54,133,97.410075,2018-02-04 08:28:17.978146,2019-05-04 02:30:11.492146,453
user_10,5438.92,123,44.218862,2018-06-29 15:38:15.855413,2019-05-11 13:07:42.624413,315
user_100,4230.10,55,76.910909,2018-12-02 14:23:12.926151,2019-02-24 13:36:50.054151,83
user_1000,843.12,42,20.074286,2019-03-27 11:03:23.105644,2019-05-05 19:08:42.183644,39


In [9]:
data = pd.merge(us_dev, trans_grouped_all, on='user_id', how='left')


In [10]:
data.head(2)

,Australia,Central_Rich_Europe,East_Europe_Balkans,Mediteranian_Europe,North_Europe_Scand,user_id,birth_year,country,city,created_date,...,num_referrals,num_successful_referrals,brand,age,total_amount,number_transactions,avg_amount,first_transaction,last_transaction,transaction_period
0,0,0,0,1,0,user_0,1992,Mediteranian_Europe,Madrid,2018-09-10 18:46:42.843855,...,0,0,1,28,6925.08,520.0,13.317462,2018-09-10 18:54:08.965855,2019-05-10 05:09:13.868855,241.0
1,0,0,1,0,0,user_1,1985,East_Europe_Balkans,Klaipeda,2018-01-11 18:53:10.531146,...,0,0,1,35,12955.54,133.0,97.410075,2018-02-04 08:28:17.978146,2019-05-04 02:30:11.492146,453.0


In [11]:
# data[['created_date', 'first_transaction', 'last_transaction']] = pd.to_datetime(data[['created_date', 'first_transaction', 'last_transaction']]) 
data['created_date'] = pd.to_datetime(data['created_date'])
data['first_transaction'] = pd.to_datetime(data['first_transaction'])
data['last_transaction'] = pd.to_datetime(data['last_transaction'])

last_trans = data['last_transaction'].max()

In [12]:
data['user_active_time'] = (last_trans - data['created_date']).dt.days
data['user_trans_time'] = (last_trans - data['first_transaction']).dt.days
data['user_trans_periodicity'] = ( data['user_trans_time'] / data['number_transactions'] ) / data['user_active_time']
data['recency'] = (last_trans - data['last_transaction']).dt.days

data['avg_amount'] = data['total_amount'] / data['user_active_time']

print(min(data.user_trans_periodicity))
data.head(2)

0.0003134717972475512


,Australia,Central_Rich_Europe,East_Europe_Balkans,Mediteranian_Europe,North_Europe_Scand,user_id,birth_year,country,city,created_date,...,total_amount,number_transactions,avg_amount,first_transaction,last_transaction,transaction_period,user_active_time,user_trans_time,user_trans_periodicity,recency
0,0,0,0,1,0,user_0,1992,Mediteranian_Europe,Madrid,2018-09-10 18:46:42.843855,...,6925.08,520.0,28.036761,2018-09-10 18:54:08.965855,2019-05-10 05:09:13.868855,241.0,247,247.0,0.001923,5.0
1,0,0,1,0,0,user_1,1985,East_Europe_Balkans,Klaipeda,2018-01-11 18:53:10.531146,...,12955.54,133.0,26.493947,2018-02-04 08:28:17.978146,2019-05-04 02:30:11.492146,453.0,489,465.0,0.007150,11.0


In [13]:
tr1 = pd.concat([pd.get_dummies(trans['transactions_type']), trans], axis=1)
tr1.head(2)

,ATM,CARD_PAYMENT,CARD_REFUND,CASHBACK,EXCHANGE,FEE,REFUND,TAX,TOPUP,TRANSFER,...,transactions_currency,amount_usd,transactions_state,ea_cardholderpresence,ea_merchant_mcc,ea_merchant_city,ea_merchant_country,direction,user_id,created_date
0,0,0,0,0,0,0,0,0,0,1,...,AED,4.55,COMPLETED,NaN,NaN,NaN,NaN,OUTBOUND,user_4368,2018-04-03 03:34:21.784487
1,0,1,0,0,0,0,0,0,0,0,...,AED,15.50,COMPLETED,FALSE,4111.0,Dubai,ARE,OUTBOUND,user_2355,2019-03-19 06:15:59.537032


In [14]:
tr2 = tr1.groupby(['user_id'], as_index=True)['ATM', 'CARD_PAYMENT', 'CARD_REFUND', 'EXCHANGE', 'REFUND', 'TAX', 'TOPUP', 'TRANSFER'].agg('sum').astype(int) 
tr2.head(2)


,ATM,CARD_PAYMENT,CARD_REFUND,EXCHANGE,REFUND,TAX,TOPUP,TRANSFER
user_id,,,,,,,,
user_0,0,333,1,5,0,0,158,23
user_1,4,78,0,0,0,0,24,27


In [15]:
def change_currency(cur):
  if cur == 'CHF':
    return 'CHF'
  elif cur == 'GBP':
    return 'GBP'
  elif cur == 'EUR':
    return 'EUR'
  else:
    return 'OTHER'

In [16]:
trans['transactions_currency'] = trans['transactions_currency'].apply(change_currency)
print(trans['transactions_currency'].value_counts())

trans = pd.concat([pd.get_dummies(trans['transactions_currency']), trans], axis=1)

trans_grouped_currency = trans.groupby(['user_id'])['CHF','EUR','GBP','OTHER'].agg('sum')

df = pd.merge(us, trans_grouped_currency, on='user_id', how='inner')

# df.info()

EUR      813234
GBP      627338
OTHER    442103
CHF       27209
Name: transactions_currency, dtype: int64


In [17]:
kk = df[['CHF', 'EUR', 'GBP', 'OTHER', 'user_id']]
kk.head(2)

,CHF,EUR,GBP,OTHER,user_id
0,0.0,25.0,0.0,495.0,user_0
1,0.0,133.0,0.0,0.0,user_1


In [18]:
data = pd.merge(data, tr2, on='user_id', how='left')
data = pd.merge(data, kk, on='user_id', how='left')

In [19]:
data.head(2)

,Australia,Central_Rich_Europe,East_Europe_Balkans,Mediteranian_Europe,North_Europe_Scand,user_id,birth_year,country,city,created_date,...,CARD_REFUND,EXCHANGE,REFUND,TAX,TOPUP,TRANSFER,CHF,EUR,GBP,OTHER
0,0,0,0,1,0,user_0,1992,Mediteranian_Europe,Madrid,2018-09-10 18:46:42.843855,...,1.0,5.0,0.0,0.0,158.0,23.0,0.0,25.0,0.0,495.0
1,0,0,1,0,0,user_1,1985,East_Europe_Balkans,Klaipeda,2018-01-11 18:53:10.531146,...,0.0,0.0,0.0,0.0,24.0,27.0,0.0,133.0,0.0,0.0


In [20]:
notif = notif[notif.status == 'SENT']
nn1 = pd.concat([pd.get_dummies(notif['reason']), notif], axis=1)
nn1.head(2)

,BLACK_FRIDAY,BLUE_TUESDAY,ENGAGEMENT_SPLIT_BILL_RESTAURANT,INVEST_IN_GOLD,JOINING_ANNIVERSARY,LOST_CARD_ORDER,MADE_MONEY_REQUEST_NOT_SPLIT_BILL,METAL_RESERVE_PLAN,NO_INITIAL_CARD_ORDER,NO_INITIAL_CARD_USE,...,PROMO_CARD_ORDER,REENGAGEMENT_ACTIVE_FUNDS,SILVER_ENGAGEMENT_FEES_SAVED,SILVER_ENGAGEMENT_INACTIVE_CARD,WELCOME_BACK,reason,channel,status,user_id,created_date
0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,REENGAGEMENT_ACTIVE_FUNDS,PUSH,SENT,user_1252,2018-12-02 17:58:33.320645
1,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,REENGAGEMENT_ACTIVE_FUNDS,PUSH,SENT,user_12783,2018-12-01 23:09:37.367127


In [21]:
nn2 = nn1.groupby(['user_id'], as_index=True)['BLACK_FRIDAY', 'BLUE_TUESDAY', 'ENGAGEMENT_SPLIT_BILL_RESTAURANT', 'INVEST_IN_GOLD',
                                              'JOINING_ANNIVERSARY', 'LOST_CARD_ORDER', 'MADE_MONEY_REQUEST_NOT_SPLIT_BILL',
                                              'METAL_RESERVE_PLAN', 'NO_INITIAL_CARD_ORDER', 'NO_INITIAL_CARD_USE',
                                              'ONBOARDING_TIPS_ACTIVATED_USERS', 'PROMO', 'PROMO_CARD_ORDER', 'REENGAGEMENT_ACTIVE_FUNDS',
                                              'WELCOME_BACK'].agg('sum').astype(int) 
nn2.head(5)


,BLACK_FRIDAY,BLUE_TUESDAY,ENGAGEMENT_SPLIT_BILL_RESTAURANT,INVEST_IN_GOLD,JOINING_ANNIVERSARY,LOST_CARD_ORDER,MADE_MONEY_REQUEST_NOT_SPLIT_BILL,METAL_RESERVE_PLAN,NO_INITIAL_CARD_ORDER,NO_INITIAL_CARD_USE,ONBOARDING_TIPS_ACTIVATED_USERS,PROMO,PROMO_CARD_ORDER,REENGAGEMENT_ACTIVE_FUNDS,WELCOME_BACK
user_id,,,,,,,,,,,,,,,
user_0,1,0,0,1,0,0,0,0,0,1,4,0,0,1,0
user_1,0,0,1,0,1,0,0,0,1,3,0,1,0,3,0
user_10,0,0,0,1,1,1,0,0,3,0,0,2,0,0,0
user_100,0,0,0,1,0,1,0,0,0,1,0,1,0,1,0
user_1000,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0


In [22]:
data = pd.merge(data, nn2, on='user_id', how='left')
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15544 entries, 0 to 15543
Data columns (total 56 columns):
Australia                                   15544 non-null uint8
Central_Rich_Europe                         15544 non-null uint8
East_Europe_Balkans                         15544 non-null uint8
Mediteranian_Europe                         15544 non-null uint8
North_Europe_Scand                          15544 non-null uint8
user_id                                     15544 non-null object
birth_year                                  15544 non-null int64
country                                     15544 non-null object
city                                        15544 non-null object
created_date                                15544 non-null datetime64[ns]
user_settings_crypto_unlocked               15544 non-null int64
plan                                        15544 non-null int64
attributes_notifications_marketing_push     10284 non-null float64
attributes_notifications_marketi

In [23]:
data.isna().sum()
data.shape

(15544, 56)

In [24]:
data = data.fillna(0)
data.isna().sum()

Australia                                   0
Central_Rich_Europe                         0
East_Europe_Balkans                         0
Mediteranian_Europe                         0
North_Europe_Scand                          0
user_id                                     0
birth_year                                  0
country                                     0
city                                        0
created_date                                0
user_settings_crypto_unlocked               0
plan                                        0
attributes_notifications_marketing_push     0
attributes_notifications_marketing_email    0
num_contacts                                0
num_referrals                               0
num_successful_referrals                    0
brand                                       0
age                                         0
total_amount                                0
number_transactions                         0
avg_amount                        

In [25]:
data.to_csv('../processed_data/training_data.csv')